In [ ]:
import os
import shutil
import sys
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.plotting import show
from bokeh.plotting import output_notebook
from bokeh.layouts import column, row, layout, gridplot
from bokeh.models import Spacer
from bokeh.models import HoverTool, ColumnDataSource
from dateutil import tz
JST = tz.gettz('Asia/Tokyo')
UTC = tz.gettz('utc')

from common import Indicators, Columns, Signal
from technical import SQUEEZER
from time_utils import TimeFilter, TimeUtils
from backtest_squeezer import Backtest

output_notebook()


def create_fig(title, size, date, format='%Y/%m/%d %H:%M'):
    TOOLS = "pan,wheel_zoom,box_zoom,box_select,crosshair,reset,save"
    TOOLTIPS=[  ( 'date',   '@date' ),
                ( 'close',  '@close{0,0}' ), 
            ]
    fig = figure(   x_axis_type="linear", # datetime
                    tools=TOOLS, 
                    plot_width=size[0],
                    plot_height=size[1],
                    tooltips=TOOLTIPS,
                    title = title
                )
    set_time(fig, date, format)
    return fig
    
def set_time(fig, date, format):
    fig.xaxis.major_label_overrides = {
                                        i: date.strftime(format) for i, date in enumerate(date)
                                        }
def plot_markers(fig, x, y, values, status, marker, color, size):
    xs = []
    ys = []
    for xx, yy, v in zip(x, y, values):
        if yy == status:
            xs.append(xx)
            ys.append(v)
    if marker == 'o':
        mark = 'circle'
    elif marker == 'v':
        mark = 'inverse_triangle'
    elif marker == '^':
        mark = 'triangle'
    elif marker == '+':
        mark = 'cross'
    elif marker == 'x':
        mark = 'x'
    fig.scatter(xs, ys, marker=mark, color=color)    

In [ ]:

def main():
    symbol = 'NSDQ'
    timeframe = 'M15'
    days = 10

    dirpath = f'./debug/squeezer/{symbol}/{timeframe}'
    os.makedirs(dirpath, exist_ok=True)
    test = Backtest(symbol, timeframe)
    test.evaluate(dirpath)
    time = test.data[Columns.JST]
    t0 = time[0]
    t1 = t0 + timedelta(days=days)

    count = 0
    while t1 < time[-1]:
        n, data = TimeUtils.slice(test.data, Columns.JST, t0, t1)   
        if n < 10:
            t0 = t1
            t1 = t0 + timedelta(days=days)
            continue
        
        jst = data[Columns.JST]
        cl = data[Columns.CLOSE]

        sqz = data[Indicators.SQUEEZER]
        std = data[Indicators.SQUEEZER_STD]
        atr = data[Indicators.SQUEEZER_ATR]
        upper = data[Indicators.SQUEEZER_UPPER]
        lower = data[Indicators.SQUEEZER_LOWER]
        signal = data[Indicators.SQUEEZER_SIGNAL]
        entry = data[Indicators.SQUEEZER_ENTRY]
        ext = data[Indicators.SQUEEZER_EXIT]

        fig1 = create_fig('', (600, 300), jst)
        fig1.line(jst, cl, color='blue', alpha=0.4)
        fig1.line(jst, upper, color='green', alpha=0.4)
        plot_markers(fig1, jst, signal, cl, 1, 'o', 'rgba(0, 255, 0, 0.2)', 10)
        plot_markers(fig1, jst, signal, cl, -1, 'o', 'rgba(255, 0, 0, 0.2)', 10)
        plot_markers(fig1, jst, entry, cl, 1, '^', 'rgba(0, 255, 0, 0.2)', 10)
        plot_markers(fig1, jst, entry, cl, -1, 'v', 'rgba(255, 0, 0, 0.2)', 10)
        
        fig2 = create_fig('', (600, 300), jst)
        fig2.line(jst, std, color='blue', alpha=0.4)
        fig2.line(jst, atr, color='red', alpha=0.4)
        
        figs = [fig1, fig2]
        l = column(*figs, background='grey')
        show(l)
        
        
        count += 1 
        t0 = t1
        t1 = t0 + timedelta(days=days)
        
        
if __name__ == '__main__':
    main()
        
